# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
import json
import pandas as pd
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
# MongoDB server
client = MongoClient("localhost", 27017)
databases = client.list_database_names()

# Access the "uk_food" database
db = client["uk_food"]

# Access the "establishments" collection
collection = db["establishments"]

# Load the data from the "establishments.json" file
with open("Resources/establishments.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Insert the data into the "establishments" collection
collection.insert_many(data)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it, Timeout: 30s, Topology Description: <TopologyDescription id: 64edc4dafe3099ddcc89a24a, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it')>]>

In [ ]:
# assign the uk_food database to a variable name
db = client["uk_food"]

In [ ]:
# review the collections in our database
collections = db.list_collection_names()

# Print the list of collections
print(collections)

In [ ]:
# assign the collection to a variable
collection = db["establishments"]

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {"hygiene": 20}

# Use count_documents to display the number of documents in the result
count = collection.count_documents(query)
print(f"Number of documents: {count}")

# Display the first document in the results using pprint
result = collection.find_one(query)
print(result)

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(collection.find(query)))

# Display the number of rows in the DataFrame
print(f"Number of rows: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
count = collection.count_documents(query)
print(f"Number of documents: {count}")

# Display the first document in the results using pprint
result = collection.find_one(query)
print(result)

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(collection.find(query)))

# Display the number of rows in the DataFrame
print(f"Number of rows: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.083812
longitude = 1.195625

query = {
    "Latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "Longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}
sort = [("hygiene", 1)]

# Print the results
results = collection.find(query).sort(sort).limit(5)
for result in results:
    print(result)

In [ ]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(list(collection.find(query).sort(sort).limit(5)))

# Display the number of rows in the DataFrame
print(f"Number of rows: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {"$match": {"hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

# Print the number of documents in the result
results = collection.aggregate(pipeline)
results = list(results)
print(f"Number of documents: {len(results)}")

# Print the first 10 results
for result in results[:10]:
    print(f"{result['_id']}: {result['count']}")
    

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))